# CS 180 Group 4: 

## Determining the Housing and Water Expenditure Based on Household-Level Data

### Introduction

The project aims to develop a regression model that can predict housing and water expenditure based on user information. The group's objective is to assist Filipino households in their financial management and provide future household heads with estimates of costs related to housing, water, and electricity, thereby aiding them in their family or house plans.

### Data Exploration

#### Preliminaries

##### Import libraries

In [ ]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import seaborn as sns

##### Import Dataset

In [ ]:
df_dataset = pd.read_csv('Family Income and Expenditure.csv')

#### Pre-processing

##### Checking the data structure

###### 
Generate Pandas Profiling

In [ ]:
import pandas_profiling as pp

*Uncomment the lines below to generate a new profile report.*

In [ ]:
# report = pp.ProfileReport(df_dataset)
# report.to_file('profile_report.html')

Refer to the detailed profile report, with the filename of 'profile_report.html', was pre-generated using the same code above.

###### 
Examine structure

In [ ]:
df_dataset.describe()

###### 
Correlation Matrix

In [ ]:
corrmat = df_dataset.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.9, square=True)
plt.show()

###### 
Correlation with House and Water Expenditure

In [ ]:
target = 'Housing and water Expenditure'
k = 12
corrmat = df_dataset.corr()
cols = corrmat.nlargest(k, target)[target].index
f, ax = plt.subplots(figsize=(10, 7))
cm = np.corrcoef(df_dataset[cols].values.T)
sns.set(font_scale=1.25)


s = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
s.set_title("Top 12 Variables Most Correlated with House and water Expenditure", size=15)
plt.show()

### Methodology

#### I. Initialization

We start the project by importing the necessary libraries and modules. Additionally, we load the FIES (Filipino Income and Expenditure) dataset and clean it by utilizing only the initially identified features and dropping rows with null values.

Dataset source: https://www.kaggle.com/datasets/grosvenpaul/family-income-and-expenditure

Furthermore, we merge the features 'Members with age less than 5 years old' and 'Members with age 5 - 17 years old' to create the 'Number of Children' feature.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from mlxtend.feature_selection import SequentialFeatureSelector

# Load the dataset
df_dataset = pd.read_csv('Family Income and Expenditure.csv')

# Initial features to use
features = [
    "Housing and water Expenditure",
    "Total Household Income",
    "Region",
    "Agricultural Household indicator",
    "Imputed House Rental Value",
    "Total Income from Entrepreneurial Acitivites",
    "Total Number of Family members",
    "Members with age less than 5 year old",
    "Members with age 5 - 17 years old",
    "Total number of family members employed",
    "Type of Building/House",
    "Type of Roof",
    "Type of Walls",
    "House Floor Area",
    "House Age",
    "Number of bedrooms",
    "Electricity",
    "Main Source of Water Supply",
    "Number of Television",
    "Number of CD/VCD/DVD",
    "Number of Component/Stereo set",
    "Number of Refrigerator/Freezer",
    "Number of Washing Machine",
    "Number of Airconditioner",
    "Number of Personal Computer"
]

# Drop features that are not in the initial set of features
df = df_dataset.filter(features, axis=1)

# To clean the dataset:
# Drop rows with missing values
df.dropna(inplace=True)

# Create feature called "Number of Children"
df['Number of Children'] = df['Members with age less than 5 year old'] + df['Members with age 5 - 17 years old']

# Drop combined features
df = df.drop(['Members with age less than 5 year old', 'Members with age 5 - 17 years old'], axis=1)

#### II. Feature Selection

Next, we will identify the list of unique categories for three categorical features: Region, Type of Building/House, and Main Source of Water Supply. Note that this step is necessary since, as we will see later, these three features are included in the final Regression Model.

In [ ]:
# Identify unique values for each of the three categorical feature included in the final regression model

# list region names
region_names = list(df['Region'].unique())
print(f'Regions: {region_names}')
print()
# output: ['CAR', 'Caraga', 'VI - Western Visayas', 'V - Bicol Region', ' ARMM', 'III - Central Luzon', 'II - Cagayan Valley', 'IVA - CALABARZON', 'VII - Central Visayas', 'X - Northern Mindanao', 'XI - Davao Region', 'VIII - Eastern Visayas', 'I - Ilocos Region', 'NCR', 'IVB - MIMAROPA', 'XII - SOCCSKSARGEN', 'IX - Zasmboanga Peninsula']

# list type of building/house
house_types = list(df['Type of Building/House'].unique())
print(f'Type of Building/House: {house_types}')
print()
# output : ['Single house', 'Duplex', 'Commercial/industrial/agricultural building', 'Multi-unit residential', 'Institutional living quarter', 'Other building unit (e.g. cave, boat)']

# list main source of water supply
water_sources = list(df['Main Source of Water Supply'].unique())
print(f'Main Source of Water Supply: {water_sources}') 
print()
# output : ['Own use, faucet, community water system', 'Shared, faucet, community water system', 'Shared, tubed/piped deep well', 'Own use, tubed/piped deep well', 'Protected spring, river, stream, etc', 'Tubed/piped shallow well', 'Lake, river, rain and others', 'Unprotected spring, river, stream, etc', 'Dug well', 'Others', 'Peddler']

Upon inspection, we can see two typographical errors. These errors are the ' ARMM' and 'IX - Zasmboanga Peninsula' categories of the Region feature. We will correct them using the code below.

In [ ]:
# Correct typos in region names
df['Region'] = df['Region'].replace(' ARMM', 'ARMM')
df['Region'] = df['Region'].replace('IX - Zasmboanga Peninsula', 'IX - Zamboanga Peninsula')

# Update region names
region_names = list(df['Region'].unique())
print(region_names)

As seen above, the typographical errors have now been collected.

Since we have categorical variables, we use target encoding to convert categorical values into numerical ones. We also store the encoding map for later use.

In [ ]:
# Define the categorical columns
cat_cols = ['Region', 'Type of Building/House', 'Type of Roof', 'Type of Walls', 'Main Source of Water Supply']

# Create an instance of the TargetEncoder from the category_encoders library
# The purpose of the TargetEncoder is to transform categorical features into numeric representations
encoder = ce.TargetEncoder(cols=cat_cols)

# Fit and transform the target encoder on the dataframe
df_encoded = encoder.fit_transform(df, df['Housing and water Expenditure'])

# Access the mapping
mapping = encoder.mapping

# Replace the original categorical columns with the target-encoded values
df[cat_cols] = df_encoded[cat_cols]

# Create a dictionary mapping the categorical columns to the target encoding
cat_dict_map = {}
for column in cat_cols:
    cat_dict_map[column] = list(mapping[column])[:-2]

From the encoding map, we create dictionaries for converting the categorical features included in the final regression model. To avoid confusion, we note that these features are identified by the feature selection method, which will be performed later on. Essentially, we added these codes to this part because we already know that they will be used.

In [ ]:
# Create a dictionary mapping the categorical columns to the target encoding

convert_region = {k: v for k, v in zip(region_names, cat_dict_map['Region'])}
convert_house_type = {k: v for k, v in zip(house_types, cat_dict_map['Type of Building/House'])}
convert_water_source = {k: v for k, v in zip(water_sources, cat_dict_map['Main Source of Water Supply'])}

Now, we are ready to begin building our regression model. We start by separating the features from the target, which is 'Housing and Water Expenditure'. Then, we determine highly correlated features and drop them. However, we can see that the initial features are not highly correlated. Thus, no features are dropped at this stage.

Link to source code:
https://stackoverflow.com/questions/29294983/how-to-calculate-correlation-between-all-columns-and-remove-highly-correlated-on

In [ ]:
# Split the dataframe into X and y
X = df.drop(['Housing and water Expenditure'], axis=1)
y = df['Housing and water Expenditure']

# Analyze the correlation among the features
threshold = 0.9
corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))
upper.head()

# Drop columns with correlations above threshold
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

if len(to_drop) > 0:
    print(f'Columns to drop: {to_drop}')
    X = X.drop(to_drop, axis=1)
else:
    print('No columns to drop. The features are not highly correlated.')
    

#### III. Training and Testing

Next, we determine the best features to use in the regression model. We split our data for training and testing. Additionally, we observe that a linear regression model was used for sequential feature selection. This is because it is the simplest regression model, and feature selection requires a significant amount of computation power and time. We drop the features that are not selected.

To do this, we will be using the class sklearn.feature_selection.SequentialFeatureSelector from the scikit-learn library that implements a sequential feature selection algorithm. This module adds or removes (in our case, we will be looking at removing features)  to form a feature subset in a greedy fashion. At each stage, this estimator chooses the best feature to add or remove based on the cross-validation score of an estimator.

More about this here:
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SequentialFeatureSelector.html

In [ ]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

# Forward feature selection
regressor = LinearRegression()
sfs = SequentialFeatureSelector(regressor, k_features='best', forward=True, scoring='neg_mean_squared_error', cv=5)
sfs.fit(X_train, y_train)

selected_features = list(X_train.columns[list(sfs.k_feature_idx_)])
print(f'Selected Features: {selected_features}')

# Drop features that are not selected
to_drop = [feature for feature in X.columns if feature not in selected_features]
X = X.drop(to_drop, axis=1)
print(f'Dropped features: {to_drop}')

#### IV. Choosing Machine Learning Model To Use

Initially, we decided to use a Linear Regression Model since it was discussed and presented clearly by our course adviser. However, we decided to test the Linear Regression Model against other models and compare their metrics.

##### Linear Regression

In [15]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test_linear = train_test_split(X, y, test_size=0.30, random_state=1)

# Forward feature selection
regressor = LinearRegression()
sfs = SequentialFeatureSelector(regressor, k_features='best', forward=True, scoring='neg_mean_squared_error', cv=5)
sfs.fit(X_train, y_train)

selected_features = list(X_train.columns[list(sfs.k_feature_idx_)])

# Drop features that are not selected
to_drop = [feature for feature in X.columns if feature not in selected_features]
X = X.drop(to_drop, axis=1)

# Fit the model and predict the target
regressor.fit(X_train, y_train)
linear_pred = regressor.predict(X_test)

# Calculate the metrics
MSE_linear = mean_squared_error(y_test_linear, linear_pred)
RMSE_linear = np.sqrt(mean_squared_error(y_test_linear, linear_pred))
MAE_linear = mean_absolute_error(y_test_linear, linear_pred)
R2_linear = r2_score(y_test_linear, linear_pred)

# Show and plot the metrics and the results
print(f'MSE: {MSE_linear}')
print(f'RMSE: {RMSE_linear}')
print(f'MAE: {MAE_linear}')
print(f'R2: {R2_linear}')

plt.scatter(y_test_linear, linear_pred)
plt.xlabel('Target')
plt.ylabel('Predicted')
plt.title('Predicted vs Target')
plt.show()


STOPPING EARLY DUE TO KEYBOARD INTERRUPT...

TypeError: 'NoneType' object is not iterable

##### Ridge Regression

In [ ]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test_ridge = train_test_split(X, y, test_size=0.30, random_state=1)

# Define the parameter grid for alpha values
param_grid = {'alpha': np.logspace(-4, 2, 50)}

ridge_model = Ridge()
grid_search = GridSearchCV(ridge_model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Print the best alpha value
print("Best alpha: ", grid_search.best_params_['alpha'])

# Evaluate the best model on the test set
ridge_best = Ridge(alpha=grid_search.best_params_['alpha'])
ridge_best.fit(X_train, y_train)
ridge_pred = ridge_best.predict(X_test)

# Calculate the metrics
MSE_ridge = mean_squared_error(y_test_ridge, ridge_pred)
RMSE_ridge = np.sqrt(mean_squared_error(y_test_ridge, ridge_pred))
MAE_ridge = mean_absolute_error(y_test_ridge, ridge_pred)
R2_ridge = r2_score(y_test_ridge, ridge_pred)

# Show and plot the metrics and the results
print(f'MSE: {MSE_ridge}')
print(f'RMSE: {RMSE_ridge}')
print(f'MAE: {MAE_ridge}')
print(f'R2: {R2_ridge}')

plt.scatter(y_test_ridge, ridge_pred)
plt.xlabel('Target')
plt.ylabel('Predicted')
plt.title('Predicted vs Target')
plt.show()

##### Lasso Regression

In [ ]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test_lasso = train_test_split(X, y, test_size=0.30, random_state=1)

# Define the parameter grid for alpha values
param_grid = {'alpha': np.logspace(-4, 2, 50)}

lasso_model = Lasso()
grid_search = GridSearchCV(lasso_model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Print the best alpha value
print("Best alpha: ", grid_search.best_params_['alpha'])

# Evaluate the best model on the test set
lasso_best = Lasso(alpha=grid_search.best_params_['alpha'])
lasso_best.fit(X_train, y_train)
lasso_pred = lasso_best.predict(X_test)

# Calculate the metrics
MSE_lasso = mean_squared_error(y_test_lasso, lasso_pred)
RMSE_lasso = np.sqrt(mean_squared_error(y_test_lasso, lasso_pred))
MAE_lasso = mean_absolute_error(y_test_lasso, lasso_pred)
R2_lasso = r2_score(y_test_lasso, lasso_pred)

# Show and plot the metrics and the results
print(f'MSE: {MSE_lasso}')
print(f'RMSE: {RMSE_lasso}')
print(f'MAE: {MAE_lasso}')
print(f'R2: {R2_lasso}')

plt.scatter(y_test_lasso, lasso_pred)
plt.xlabel('Target')
plt.ylabel('Predicted')
plt.title('Predicted vs Target')
plt.show()

##### Polynomial Regression

In [ ]:
# Account for the interaction between features using a polynomial of degree 2
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_interactions = poly.fit_transform(X)

# Split the data into train and test sets using X_interactions instead of X
X_train, X_test, y_train, y_test_poly = train_test_split(X_interactions, y, test_size=0.30, random_state=1)

# Fit the model and predict the target
regressor.fit(X_train, y_train)
poly_pred = regressor.predict(X_test)

# Calculate the metrics
MSE_poly = mean_squared_error(y_test_poly, poly_pred)
RMSE_poly = np.sqrt(mean_squared_error(y_test_poly, poly_pred))
MAE_poly = mean_absolute_error(y_test_poly, poly_pred)
R2_poly = r2_score(y_test_poly, poly_pred)

# Show and plot the metrics and the results
print(f'MSE: {MSE_poly}')
print(f'RMSE: {RMSE_poly}')
print(f'MAE: {MAE_poly}')
print(f'R2: {R2_poly}')

plt.scatter(y_test_poly, poly_pred)
plt.xlabel('Target')
plt.ylabel('Predicted')
plt.title('Predicted vs Target')
plt.show()

##### Neural Network Model

This Neural Network model uses all of the columns as opposed to selected features to take advantage of the properties of neural networks and the amount of data.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from mlxtend.feature_selection import SequentialFeatureSelector
import matplotlib.pyplot as plt
import category_encoders as ce
import tensorflow as tf
import keras as keras
from keras.models import Sequential
from keras.layers import Dense

# Load the dataset
df_dataset = pd.read_csv('Family Income and Expenditure.csv')

# Identify the list of unique categories for the categorical features
region = list(df_dataset['Region'].unique())
main_source_of_income = list(df_dataset['Main Source of Income'].unique())
household_head_sex = list(df_dataset['Household Head Sex'].unique())
household_head_marital_status = list(df_dataset['Household Head Marital Status'].unique())
household_head_highest_grade_completed = list(df_dataset['Household Head Highest Grade Completed'].unique())
household_head_job_or_business_indicator = list(df_dataset['Household Head Job or Business Indicator'].unique())
household_head_occupation = list(df_dataset['Household Head Occupation'].unique())
household_head_class_of_worker = list(df_dataset['Household Head Class of Worker'].unique())
type_of_household = list(df_dataset['Type of Household'].unique())
type_of_buildingHouse = list(df_dataset['Type of Building/House'].unique())
type_of_roof = list(df_dataset['Type of Roof'].unique())
type_of_walls = list(df_dataset['Type of Walls'].unique())
tenure_status = list(df_dataset['Tenure Status'].unique())
toilet_facilities = list(df_dataset['Toilet Facilities'].unique())
main_source_of_water_supply = list(df_dataset['Main Source of Water Supply'].unique())

# correct typos in region names
df_dataset['Region'] = df_dataset['Region'].replace(' ARMM', 'ARMM')
df_dataset['Region'] = df_dataset['Region'].replace('IX - Zasmboanga Peninsula', 'IX - Zamboanga Peninsula')

# update region names
region = list(df_dataset['Region'].unique())

# Define the categorical columns
cat_cols = ['Region', 'Main Source of Income', 'Household Head Sex', 'Household Head Marital Status', 'Household Head Highest Grade Completed', 'Household Head Job or Business Indicator', 'Household Head Occupation', 'Household Head Class of Worker', 'Type of Household', 'Type of Building/House', 'Type of Roof', 'Type of Walls', 'Tenure Status', 'Toilet Facilities', 'Main Source of Water Supply', ]

# Create an instance of the TargetEncoder
encoder = ce.TargetEncoder(cols=cat_cols)

# Fit and transform the target encoder on the dataframe
df_encoded = encoder.fit_transform(df_dataset, df_dataset['Housing and water Expenditure'])

# Access the mapping
mapping = encoder.mapping

# Replace the original categorical columns with the target-encoded values
df_dataset[cat_cols] = df_encoded[cat_cols]

# Create a dictionary mapping the categorical columns to the target encoding
cat_dict_map = {}
for column in cat_cols:
    cat_dict_map[column] = list(mapping[column])[:-2]

convert_region = {k: v for k, v in zip(region, cat_dict_map['Region'])}
convert_main_source_of_income = {k: v for k, v in zip(main_source_of_income, cat_dict_map['Main Source of Income'])}
convert_household_head_sex = {k: v for k, v in zip(household_head_sex, cat_dict_map['Household Head Sex'])}
convert_household_head_marital_status = {k: v for k, v in zip(household_head_marital_status, cat_dict_map['Household Head Marital Status'])}
convert_household_head_highest_grade_completed = {k: v for k, v in zip(household_head_highest_grade_completed, cat_dict_map['Household Head Highest Grade Completed'])}
convert_household_head_job_or_business_indicator = {k: v for k, v in zip(household_head_job_or_business_indicator, cat_dict_map['Household Head Job or Business Indicator'])}
convert_household_head_occupation = {k: v for k, v in zip(household_head_occupation, cat_dict_map['Household Head Occupation'])}
convert_household_head_class_of_worker = {k: v for k, v in zip(household_head_class_of_worker, cat_dict_map['Household Head Class of Worker'])}
convert_type_of_household = {k: v for k, v in zip(type_of_household, cat_dict_map['Type of Household'])}
convert_type_of_buildingHouse = {k: v for k, v in zip(type_of_buildingHouse, cat_dict_map['Type of Building/House'])}
convert_type_of_roof = {k: v for k, v in zip(type_of_roof, cat_dict_map['Type of Roof'])}
convert_type_of_walls = {k: v for k, v in zip(type_of_walls, cat_dict_map['Type of Walls'])}
convert_tenure_status = {k: v for k, v in zip(tenure_status, cat_dict_map['Tenure Status'])}
convert_toilet_facilities = {k: v for k, v in zip(toilet_facilities, cat_dict_map['Toilet Facilities'])}
convert_main_source_of_water_supply = {k: v for k, v in zip(main_source_of_water_supply, cat_dict_map['Main Source of Water Supply'])}

# columns = ['Housing and water Expenditure', 'Total Household Income', 'Region', 'Imputed House Rental Value', 'Total Number of Family members', 'Type of Building/House', 'House Floor Area', 'House Age', 'Number of bedrooms', 'Main Source of Water Supply', 'Number of CD/VCD/DVD', 'Number of Component/Stereo set', 'Number of Refrigerator/Freezer', 'Number of Washing Machine', 'Number of Airconditioner', 'Number of Personal Computer']
# df_dataset = df_dataset.filter(columns, axis=1)

# Split into input (X) and output (y) variables
X = df_dataset.drop(['Housing and water Expenditure'], axis=1)
y = df_dataset['Housing and water Expenditure']

X_train, X_temp, Y_train, Y_temp = train_test_split(X, y, test_size = 0.3, random_state=2)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size = 0.5, random_state=2)

# define the keras model
model = Sequential()
model.add(Dense(12, input_shape=(59,), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# compile the keras model
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=[tf.keras.metrics.MeanSquaredError(), tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanAbsoluteError()])

*Uncomment the lines below to retrain the neural network model.*

In [ ]:
# # Fit the keras model on the dataset
# model.fit(X_train, Y_train, validation_data = (X_val, Y_val), epochs=1000)

Since training the dataset takes a lot of time, a pre-trained model will be saved to save time.<br>
*Uncomment the lines below to retrain the neural network model.*<br>
***Do not save unless the model has been retrained***

In [ ]:
# # Save the Neural Network model into a file
# model.save("model_NeuralNetwork.keras")

Finally, we assess the metrics, such as Mean Square Error (MSE), Root Mean Square Error (RMSE), Mean Absolute Error (MAE). We can see that the Neural Network model has a relatively low MAE and a relatively low RMSE, indicating that it performs well in predicting the target. Note that although the RMSE is in the thousands, its unit is the Philippine Peso.

In [ ]:
# Load the Neural Network model from a file
loaded = loaded_model = tf.keras.saving.load_model("model_NeuralNetwork.keras")

# Evaluating the Neural Network model
_, MSE_neural, RMSE_neural, MAE_neural = loaded.evaluate(X_test, Y_test)

Y_pred = loaded.predict(X_test)

# Show and plot the metrics and the results
print(f'MSE: {MSE_neural}')
print(f'RMSE: {RMSE_neural}')
print(f'MAE: {MAE_neural}')

# Plot Predicted vs Target
plt.scatter(Y_pred, Y_test)
plt.xlabel('Target')
plt.ylabel('Predicted')
plt.title('Predicted vs Target')
plt.show()

##### Comparing the Metrics of the Models

In [ ]:
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, sharex=True, sharey=True, figsize=(15, 20))
fig.suptitle('Predicted vs Target')

ax1.set_title('Linear Regression')
ax1.scatter(y_test_linear, linear_pred)

ax2.set_title('Ridge')
ax2.scatter(y_test_ridge, ridge_pred)

ax3.set_title('Lasso')
ax3.scatter(y_test_lasso, lasso_pred)

ax4.set_title('Polynomial Regression')
ax4.scatter(y_test_poly, poly_pred)

ax5.set_title('Neural Network')
ax5.scatter(Y_test, Y_pred)

ax6.axis('off')

for ax in fig.get_axes():
    ax.label_outer()


metrics = {
    'Model': ['Linear Regression', 'Ridge', 'Lasso', 'Polynomial Regression', 'Neural Network'],
    'MSE': [MSE_linear, MSE_ridge, MSE_lasso, MSE_poly, MSE_neural],
    'RMSE': [RMSE_linear, RMSE_ridge, RMSE_lasso, RMSE_poly, RMSE_neural],
    'MAE': [MAE_linear, MAE_ridge, MAE_lasso, MAE_poly, MAE_neural],
    'R2': [R2_linear, R2_ridge, R2_lasso, R2_poly, ""]
    }

df_metrics = pd.DataFrame(data=metrics)
pd.set_option('display.float_format', '{:.10f}'.format)
df.style \
  .format(precision=9)
df_metrics


Based on the metrics and graph presented, it can be observed that the Neural Network Model exhibits superior performance. However, it should be noted that neural networks, or deep learning models heavily relies on data, necessitating a substantial volume of data to effectively learn the underlying structure and parameters. We believe that the dataset used in this study may not be of sufficient size to establish the reliability of the Neural Network Model. Moreover, neural networks have a tendency to overfit, compromising their predictive capabilities for future instances.

In the end, we decided to use the Polynomial Regression Model (2nd Degree). Firstly, this is because Regression Analysis is less of a black box and is easier to communicate.
Two important factors we considered when we chose a model are how simple the model is and how interpretable it is. A simpler model means it’s easier to communicate how the model itself works and how to interpret the results of a model. And based on the metrics and the graph, the Polynomial Regression Model outperforms the other models except the Neural Network Model.

#### V. Polynomial Regression Model 2nd Degree

Now, we take into account the interactions between the features. Particualarly, we model these interactions as a second-degree polynomial. Apart from saving computational resources, using this degree yields the best results compared to using higher degrees. This can be explained by factors such as overfitting. We also split the data again for the training and testing of our Polynomial Regression model. It is also important to note that it now uses X_interactions, which account for the feature interactions, instead of the original data X.

In [ ]:
# Polynomial Regression
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_interactions = poly.fit_transform(X)

# Split the data into train and test sets using X_interactions instead of X
X_train, X_test, y_train, y_test = train_test_split(X_interactions, y, test_size=0.30, random_state=1)

# Fit the model and predict the target
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

Finally, we assess the metrics, such as RMSE and R2. We can see that our Polynomial Regression model has an R2 close to 1 and a relatively low RMSE, indicating that it performs well in predicting the target. Note that although the RMSE is in the thousands, its unit is the Philippine Peso.

In [ ]:
# Show and plot the results
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, y_pred))}')
print(f'R2: {r2_score(y_test, y_pred)}')

plt.scatter(y_test, y_pred)
plt.xlabel('Target')
plt.ylabel('Predicted')
plt.title('Predicted vs Target')
plt.show()

#### VI. Creating a Web App For the Practical Use of the Model

Using Flask, we now make a locally-hosted Web App that predicts the Housing and Water expenditures.

In [ ]:
# Create a function to predict the Housing and Water expenditure given an array of user inputs
def model_predict(input_arr):
    encoded_input = [input_arr]

    # Encode categorical variables
    for i in range(len(input_arr)):
        if i == 1:
            encoded_input[0][i] = convert_region[input_arr[i]]
        elif i == 4:
            encoded_input[0][i] = convert_house_type[input_arr[i]]
        elif i == 8:
            encoded_input[0][i] = convert_water_source[input_arr[i]] 

    # Perform polynomial feature transformation if applicable
    input_interactions = poly.fit_transform(encoded_input)

    # Make prediction
    prediction = regressor.predict(input_interactions)

    # Return prediction
    return float(prediction[0])

In [ ]:
# Import libraries
from flask import Flask, render_template, request, jsonify

# Change based on Saved location
path = r'C:\Users\Brylle\Desktop\CS 180 Project\templates'

app = Flask(__name__, static_folder = path)

@app.route('/')
def index():
    return render_template('index.html')

# Flask route handler
@app.route('/calculate', methods=['POST'])
def calculate():
    features = [key for key in request.form.keys() if key.startswith('input')]
    input_arr = [request.form.get(key) for key in features]
    conv_input = [float(input_arr[i]) if i not in (1,4,8) else input_arr[i] for i in range(len(input_arr))]
    result = 'Php'+str(round(model_predict(conv_input), 2))
    return jsonify(result=result)

if __name__ == '__main__':
    app.run(host='0.0.0.0')